In [2]:
# Import numpy for array operations
import numpy as np

### Image Augmentation Example

In [22]:
# Import ImageDataGenerator for image preprocessing/augmentation
# This basically creates multiple copies of train images by jittering(adding noise). 
# This includes rotating, zooming in, flipping, shifting, etc.
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest') # 'nearest' is kind of algorithm to fill pixel values while transformation

### Save augmented images 

In [23]:
import os
from keras.preprocessing.image import img_to_array, load_img

img = load_img('C:\\Users\\venup\\Documents\\TShirt Graphic Classification\\Images_Master\\Train\\Abstract\\img_24.png')  # this is a PIL image
x = img_to_array(img)  # this is a Numpy array with shape (480, 640, 3)
x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 480, 640, 3)

# Create a directory named 'preview' in which we can save augmented images. 
os.system('mkdir preview')

# the .flow() command below generates batches of randomly transformed images
# and saves the results to the `preview/` directory
i = 0
for batch in datagen.flow(x, batch_size=1,
                          save_to_dir='preview', save_prefix='c0', save_format='jpg'):
    i += 1
    if i > 20:
        break  # otherwise the generator would loop indefinitely

### Model Architecture

In [4]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(150, 150, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(23))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

### Define train and validation generators
This is extremely useful when the samples are arranged in a structure, where all samples corresponding to one class are in a single folder. And all such folders corresponding to train/validation are in a parent folder. 

In [5]:
batch_size = 50

# This is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

# This is the augmentation configuration we will use for testing:
# Only rescaling. Other transformations are not required for testing. Duh!
test_datagen = ImageDataGenerator(rescale=1./255)

# this is a generator that will read pictures found in
# subfolers of 'data/train', and indefinitely generate
# batches of augmented image data
train_generator = train_datagen.flow_from_directory(
        'Images_Master//Train',  # this is the target directory
        target_size=(150, 150),  # all images will be resized to 150x150
        batch_size=batch_size,
        class_mode='categorical')  # since we use binary_crossentropy loss, we need binary labels

# this is a similar generator, for validation data
validation_generator = test_datagen.flow_from_directory(
        'Images_Master//Validation',
        target_size=(150, 150),
        batch_size=batch_size,
       class_mode='categorical')

Found 4285 images belonging to 23 classes.
Found 1618 images belonging to 23 classes.


### Train the model

In [6]:
# fit_generator is similar to 'fit'. But instead of x_train and y_train, we pass 'train_generator' 
# which already has the samples and their corresponding target information.

# 'step' here is one mini-batch
# 'steps_per_epoch' is number of batches per epoch.
# Typically 'steps_per_epoch' should be total_samples divided by batch_size
model.fit_generator(
        train_generator,
        steps_per_epoch=4285//batch_size, # '//' in python returns only the quotient
        epochs=5,
        validation_data=validation_generator,
        validation_steps=1618//batch_size)
model.save_weights('first_try.h5')  # a

#always save your weights after training or during training

Epoch 1/5
85/85 [==============================] - 731s 9s/step - loss: 2.3277 - acc: 0.3301 - val_loss: 2.1776 - val_acc: 0.3156
Epoch 2/5
85/85 [==============================] - 661s 8s/step - loss: 1.9280 - acc: 0.4830 - val_loss: 2.2306 - val_acc: 0.3259
Epoch 3/5
85/85 [==============================] - 750s 9s/step - loss: 1.7972 - acc: 0.5367 - val_loss: 2.2997 - val_acc: 0.3355
Epoch 4/5
85/85 [==============================] - 854s 10s/step - loss: 1.7264 - acc: 0.5633 - val_loss: 2.6818 - val_acc: 0.3265
Epoch 5/5
85/85 [==============================] - 811s 10s/step - loss: 1.6681 - acc: 0.5792 - val_loss: 2.6496 - val_acc: 0.3170


In [7]:
# print(train_generator.filenames)
print(train_generator.class_indices) # This returns class labels of directories

{'Abstract': 0, 'Biker': 1, 'Camouflage': 2, 'Checked': 3, 'Colourblocked': 4, 'Conversational': 5, 'Floral': 6, 'Geometric': 7, 'Graphic': 8, 'Humour and Comic': 9, 'Music': 10, 'People and Places': 11, 'Polka Dots': 12, 'Self Design': 13, 'Solid': 14, 'Sports': 15, 'Sports and Team Jersey': 16, 'Striped': 17, 'Superhero': 18, 'Tie and Dye': 19, 'Tribal': 20, 'Typography': 21, 'Varsity': 22}


### Test the model

In [8]:
test_generator = test_datagen.flow_from_directory(
        'Images_Master//Test',  # this is the target directory
        target_size=(150, 150),  # all images will be resized to 150x150
        batch_size=batch_size,
        class_mode='categorical')  # since we use binary_crossentropy loss, we need binary labels

Found 2849 images belonging to 23 classes.


In [16]:
test_prob = model.predict_generator(test_generator, steps=len(test_generator)) # this returns the probabilities
test_pred_classes = np.argmax(test_prob, axis=1) # convert probabilities to classes
print(test_pred_classes)


[14 21 21 ... 14 14 14]


In [17]:
len(test_generator)

57

In [18]:
# Check the corresponding filenames of the predictions
print(test_generator.filenames[:])

['Abstract\\img_1001.png', 'Abstract\\img_1027.png', 'Abstract\\img_1030.png', 'Abstract\\img_1078.png', 'Abstract\\img_1107.png', 'Abstract\\img_1141.png', 'Abstract\\img_115.png', 'Abstract\\img_1153.png', 'Abstract\\img_1158.png', 'Abstract\\img_1166.png', 'Abstract\\img_1185.png', 'Abstract\\img_1213.png', 'Abstract\\img_1214.png', 'Abstract\\img_126.png', 'Abstract\\img_1271.png', 'Abstract\\img_1283.png', 'Abstract\\img_1304.png', 'Abstract\\img_1308.png', 'Abstract\\img_1310.png', 'Abstract\\img_1313.png', 'Abstract\\img_1363.png', 'Abstract\\img_1379.png', 'Abstract\\img_1409.png', 'Abstract\\img_1441.png', 'Abstract\\img_1447.png', 'Abstract\\img_1468.png', 'Abstract\\img_1531.png', 'Abstract\\img_1533.png', 'Abstract\\img_154.png', 'Abstract\\img_1589.png', 'Abstract\\img_1595.png', 'Abstract\\img_1608.png', 'Abstract\\img_1616.png', 'Abstract\\img_1622.png', 'Abstract\\img_1630.png', 'Abstract\\img_1694.png', 'Abstract\\img_1712.png', 'Abstract\\img_1723.png', 'Abstract\\img

### Save test predictions

In [19]:
print(len(test_prob))

2849


In [21]:
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score
#Prediction metrics.
print(confusion_matrix(y_pred=test_pred_classes, y_true=test_generator.classes))
print(accuracy_score(y_pred=test_pred_classes, y_true=test_generator.classes))
print(classification_report(y_pred=test_pred_classes, y_true=test_generator.classes))

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0  55   0   0   5
    0   0   0  50   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   6   0   0   3
    0   0   0   8   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   1   0   0   0
    0   0   0   4   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   2   0   0   0
    0   0   0   2   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0  60   0   0   4
    0   0   0  37   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0  21   0   0   5
    0   0   0  34   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0  32   0   0   1
    0   0   0  25   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0  34   0   0   7
    0   0   0  39   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0  69   0   0  17
    0   0   0  60   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   6   0   0   2
    0   0   0   4   0]
 [  0   0   0   0   0   0   0 

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
